**Principal Component Analysis**

You will implement dimensionality reduction with PCA.  

1). Read iris_dataset.csv (4 features, hence 4 PCs)

2). Find the principal components

3). Recontruct the dataset (X_hat)

4). Determine the accuracy of X_hat for 1 PC and 4 PCs using LDA classifier (provided below)


In [7]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from numpy import linalg as LA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score


# Load data - 150 observations, 4 features, 3 classes, 
df = pd.read_csv("iris_dataset.csv", header=None)
print(df.describe())
data = df.values
print(np.shape(data))

                0           1           2           3           4
count  150.000000  150.000000  150.000000  150.000000  150.000000
mean     5.843333    3.057333    3.758000    1.199333    2.000000
std      0.828066    0.435866    1.765298    0.762238    0.819232
min      4.300000    2.000000    1.000000    0.100000    1.000000
25%      5.100000    2.800000    1.600000    0.300000    1.000000
50%      5.800000    3.000000    4.350000    1.300000    2.000000
75%      6.400000    3.300000    5.100000    1.800000    3.000000
max      7.900000    4.400000    6.900000    2.500000    3.000000
(150, 5)


In [8]:
## Setup

# Shuffle data randomly
shuffled_data = data;
np.random.shuffle(shuffled_data)
X = shuffled_data[:,0:4]  # 150x4
y = shuffled_data[:,4]

# Classification accuracy with the original dataset using LDA
model_mean_scores = []
model = LinearDiscriminantAnalysis().fit(X, y)
scores = cross_val_score(model, X, y, cv=10)
model_mean_scores.append(np.mean(scores))
print('>> Average accuracy with the original dataset = {0:0.4f}'.format(model_mean_scores[0]))


>> Average accuracy with the original dataset = 0.9800


In [9]:
def evaluate_accuracy(X_hat, Num_PC, y):
    
  ###############################################
  # Evaluate classificatin accuracy with LDA
  ###############################################
  '''
    Inputs:
      X_hat: reconstructed dataset. dimension=150x4
      Num_PC: number of PC's used to recover X_hat
      y: class label vector. dimension=150x1

  '''
  
  from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
  from sklearn.model_selection import cross_val_score

  X_train = X_hat[:,0:Num_PC]        # dimensionally reduced dataset
  y_train = y
  recon_error = (np.linalg.norm((X-X_hat),'fro')) / (np.linalg.norm(X, 'fro'))

  model_mean_scores = []
  model = LinearDiscriminantAnalysis().fit(X_train, y_train)
  scores = cross_val_score(model, X_train, y_train, cv=10)
  model_mean_scores.append(np.mean(scores))

  print('Average accuracy = {0:0.4f} with {1:1d} PCs'
     .format(model_mean_scores[0], Num_PC))
  print('Reconstruction error = {0:0.6f} with {1:1d} PCs'.format(recon_error,Num_PC, model_mean_scores[0]))

In [10]:
### Your code goes here ...
def PCA(X, Num_PC):
  u = [np.mean(X[:,0]), np.mean(X[:,1]), np.mean(X[:,2]), np.mean(X[:,3])]

  XM = np.subtract(X,u)
  cov = np.cov(XM.T)
  w, Phi = LA.eig(cov)
  index = np.argsort(w)[::-1]
  w = w[index]
  Phi = Phi[:,index]
  
  PC = Phi[:,0:Num_PC]
  Y = np.dot(XM,PC)
  Xhat = np.add(np.dot(Y,PC.T),u)

  return Phi, Xhat

## Use function evaluate_accuracy 
PC, X_hat = PCA(X,1)
evaluate_accuracy(X_hat, 1, y)  # classification accuracy with 1 PC
PC, X_hat = PCA(X,4)
evaluate_accuracy(X_hat, 4, y)  # classification accuracy with 4 PCs


Average accuracy = 0.9267 with 1 PCs
Reconstruction error = 0.073378 with 1 PCs
Average accuracy = 0.9800 with 4 PCs
Reconstruction error = 0.000000 with 4 PCs
